In [11]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import logging
import warnings
warnings.filterwarnings("ignore")

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Load datasets
data_path = "data"
file_names = {
    "bitcoin": "bitcoin.csv",
    "gold": "gold.csv",
    "google_trends": "google_trends.csv",
    "sp500": "sp500.csv",
    "treasury_3m": "treasury_3m.csv",
    "treasury_10y": "treasury_10y.csv",
    "copper": "copper.csv",
    "oil": "oil.csv",
    "unemployment": "unemployment.csv",
    "moex": "MOEX.csv",
    "sse": "SSE.csv",
    "stoxx": "STOXX_600.csv",
    "vix": "vix.csv",
    "spgsci": "spgsci.csv"
}

data = {}
for key, file in file_names.items():
    file_path = os.path.join(data_path, file)
    if os.path.exists(file_path):
        logging.info(f"Loading {file}")
        df = pd.read_csv(file_path)
        if "timestamp" not in df.columns:
            logging.warning(f"{file} skipped: no 'timestamp' column")
            continue
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df.replace({'.': np.nan}, inplace=True)
        for col in df.columns:
            if col != "timestamp":
                df[col] = pd.to_numeric(df[col], errors='coerce')
        data[key] = df
    else:
        logging.warning(f"{file} not found")

# Rename columns
rename_map = {
    "bitcoin": {"Close": "bitcoin_close", "Open": "bitcoin_open", "High": "bitcoin_high", "Low": "bitcoin_low", "Volume": "bitcoin_volume"},
    "gold": {"Close": "gold_close", "Open": "gold_open", "High": "gold_high", "Low": "gold_low", "Volume": "gold_volume"},
    "oil": {"Close": "oil_close", "Open": "oil_open", "High": "oil_high", "Low": "oil_low", "Volume": "oil_volume"},
    "copper": {"price": "copper_price"},
    "google_trends": {"SPX": "google_spx", "ETF": "google_etf", "index fund": "google_index_fund", "sp500": "google_sp500"},
    "unemployment": {"Unemployment": "unemployment_rate"},
    "treasury_3m": {"Close": "treasury_3m"},
    "treasury_10y": {"Close": "treasury_10y"},
    "sp500": {"Close": "sp500_close"},
    "moex": {"Close": "moex_close"},
    "sse": {"Close": "sse_close"},
    "stoxx": {"Close": "stoxx_close"},
    "vix": {"Close": "vix_close"},
    "spgsci": {"Close": "spgsci_close"}
}

for key, renames in rename_map.items():
    if key in data:
        data[key] = data[key].rename(columns=renames)

# Merge and clean data
logging.info("Merging all dataframes")
sp500 = data["sp500"]
all_data = sp500[["timestamp", "sp500_close"]]
for key, df in data.items():
    if key != "sp500":
        all_data = all_data.merge(df, on="timestamp", how="left")

all_data.sort_values("timestamp", inplace=True)
all_data.fillna(method="ffill", inplace=True)
all_data.dropna(inplace=True)
all_data = all_data.loc[:, all_data.nunique() > 1]
logging.info("Merged dataset shape: %s", all_data.shape)

# Define target and features
target = "sp500_close"
exclude_cols = ["timestamp", target]
exog_vars = [col for col in all_data.columns if col not in exclude_cols]

# Monthly SARIMAX backtest with reduced grid
results = []
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2025-03-01")
current_date = start_date

while current_date <= end_date:
    logging.info(f"Backtest for month starting {current_date.strftime('%Y-%m-%d')}")
    next_month = current_date + pd.DateOffset(months=1)

    train = all_data[all_data["timestamp"] < current_date]
    test = all_data[(all_data["timestamp"] >= current_date) & (all_data["timestamp"] < next_month)]

    if len(train) < 100 or len(test) < 10:
        logging.warning("Skipping month due to insufficient data")
        current_date += pd.DateOffset(months=1)
        continue

    y_train = train[target]
    y_test = test[target]
    X_train = train[exog_vars]
    X_test = test[exog_vars]

    # Standardize and apply PCA
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    # SARIMAX parameter tuning with reduced grid
    best_aic = np.inf
    best_model = None
    best_order = None
    best_seasonal_order = None

    for p in [0, 1]:
        for q in [0, 1]:
            for P in [0, 1]:
                for Q in [0, 1]:
                    try:
                        model = SARIMAX(
                            y_train,
                            exog=X_train_pca,
                            order=(p, 1, q),
                            seasonal_order=(P, 0, Q, 12),
                            enforce_stationarity=False,
                            enforce_invertibility=False
                        )
                        model_fit = model.fit(disp=False)
                        if model_fit.aic < best_aic:
                            best_aic = model_fit.aic
                            best_model = model_fit
                            best_order = (p, 1, q)
                            best_seasonal_order = (P, 0, Q, 12)
                    except Exception as e:
                        logging.debug(f"Model ({p},{1},{q}) x ({P},0,{Q},12) failed: {e}")
                        continue

    if best_model:
        logging.info(f"Best model: order={best_order}, seasonal_order={best_seasonal_order}, AIC={best_aic:.2f}")
        forecast = best_model.forecast(steps=len(X_test_pca), exog=X_test_pca)
        r2 = r2_score(y_test, forecast)
        mae = mean_absolute_error(y_test, forecast)
        rmse = np.sqrt(mean_squared_error(y_test, forecast))

        results.append({
            "Backtest": len(results) + 1,
            "Train Size": len(train),
            "Test Start": current_date.strftime("%Y-%m-%d"),
            "R2": r2,
            "MAE": mae,
            "RMSE": rmse,
            "Order": best_order,
            "Seasonal Order": best_seasonal_order,
            "AIC": best_aic
        })

    current_date += pd.DateOffset(months=1)

# Save and print results
results_df = pd.DataFrame(results)
os.makedirs("results", exist_ok=True)
results_df.to_csv("results/sarimax_backtest_auto_reduced.csv", index=False)
logging.info("Saved results to results/sarimax_backtest_auto_reduced.csv")
print(results_df)


2025-04-14 21:55:54,324 - INFO - Loading bitcoin.csv
2025-04-14 21:55:54,329 - INFO - Loading gold.csv
2025-04-14 21:55:54,332 - INFO - Loading google_trends.csv
2025-04-14 21:55:54,336 - INFO - Loading sp500.csv
2025-04-14 21:55:54,339 - INFO - Loading treasury_3m.csv
2025-04-14 21:55:54,342 - INFO - Loading treasury_10y.csv
2025-04-14 21:55:54,345 - INFO - Loading copper.csv
2025-04-14 21:55:54,350 - INFO - Loading oil.csv
2025-04-14 21:55:54,352 - INFO - Loading unemployment.csv
2025-04-14 21:55:54,357 - INFO - Loading MOEX.csv
2025-04-14 21:55:54,360 - INFO - Loading SSE.csv
2025-04-14 21:55:54,363 - INFO - Loading STOXX_600.csv
2025-04-14 21:55:54,365 - INFO - Loading vix.csv
2025-04-14 21:55:54,367 - INFO - Loading spgsci.csv
2025-04-14 21:55:54,371 - INFO - Merging all dataframes
2025-04-14 21:55:54,384 - INFO - Merged dataset shape: (801, 41)
2025-04-14 21:55:54,385 - INFO - Backtest for month starting 2023-01-01
2025-04-14 21:55:59,522 - INFO - Best model: order=(0, 1, 1), sea

    Backtest  Train Size  Test Start        R2        MAE       RMSE  \
0          1         250  2023-01-01 -0.036554   6.774218   7.968159   
1          2         270  2023-02-01  0.271854   5.044638   5.649179   
2          3         289  2023-03-01  0.031712   4.781434   5.788832   
3          4         312  2023-04-01  0.231874   2.042358   2.613261   
4          5         331  2023-05-01 -3.421553   6.177315   8.220778   
5          6         353  2023-06-01 -5.406618  12.678757  13.768809   
6          7         374  2023-07-01 -1.190491   8.484095   9.034702   
7          8         394  2023-08-01  0.531595   2.840536   3.711495   
8          9         417  2023-09-01  0.423780   5.371826   6.579845   
9         10         437  2023-10-01  0.527804   4.450619   5.237092   
10        11         459  2023-11-01  0.063884   8.934425   9.568126   
11        12         480  2023-12-01 -0.449804   7.479950   8.989052   
12        13         500  2024-01-01 -0.368995   6.788105   8.47

In [2]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import logging
import warnings
warnings.filterwarnings("ignore")

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Load datasets
data_path = "data"
file_names = {
    "bitcoin": "bitcoin.csv",
    "gold": "gold.csv",
    "google_trends": "google_trends.csv",
    "sp500": "sp500.csv",
    "treasury_3m": "treasury_3m.csv",
    "treasury_10y": "treasury_10y.csv",
    "copper": "copper.csv",
    "oil": "oil.csv",
    "unemployment": "unemployment.csv",
    "moex": "MOEX.csv",
    "sse": "SSE.csv",
    "stoxx": "STOXX_600.csv",
    "vix": "vix.csv",
    "spgsci": "spgsci.csv"
}

data = {}
for key, file in file_names.items():
    file_path = os.path.join(data_path, file)
    if os.path.exists(file_path):
        logging.info(f"Loading {file}")
        df = pd.read_csv(file_path)
        if "timestamp" not in df.columns:
            logging.warning(f"{file} skipped: no 'timestamp' column")
            continue
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df.replace({'.': np.nan}, inplace=True)
        for col in df.columns:
            if col != "timestamp":
                df[col] = pd.to_numeric(df[col], errors='coerce')
        data[key] = df
    else:
        logging.warning(f"{file} not found")

rename_map = {
    "bitcoin": {"Close": "bitcoin_close", "Open": "bitcoin_open", "High": "bitcoin_high", "Low": "bitcoin_low", "Volume": "bitcoin_volume"},
    "gold": {"Close": "gold_close", "Open": "gold_open", "High": "gold_high", "Low": "gold_low", "Volume": "gold_volume"},
    "oil": {"Close": "oil_close", "Open": "oil_open", "High": "oil_high", "Low": "oil_low", "Volume": "oil_volume"},
    "copper": {"price": "copper_price"},
    "google_trends": {"SPX": "google_spx", "ETF": "google_etf", "index fund": "google_index_fund", "sp500": "google_sp500"},
    "unemployment": {"Unemployment": "unemployment_rate"},
    "treasury_3m": {"Close": "treasury_3m"},
    "treasury_10y": {"Close": "treasury_10y"},
    "sp500": {"Close": "sp500_close"},
    "moex": {"Close": "moex_close"},
    "sse": {"Close": "sse_close"},
    "stoxx": {"Close": "stoxx_close"},
    "vix": {"Close": "vix_close"},
    "spgsci": {"Close": "spgsci_close"}
}

for key, renames in rename_map.items():
    if key in data:
        data[key] = data[key].rename(columns=renames)

logging.info("Merging all dataframes")
sp500 = data["sp500"]
all_data = sp500[["timestamp", "sp500_close"]]
for key, df in data.items():
    if key != "sp500":
        all_data = all_data.merge(df, on="timestamp", how="left")

all_data.sort_values("timestamp", inplace=True)
all_data.fillna(method="ffill", inplace=True)
all_data.dropna(inplace=True)

def add_lag_rolling_features(df, base_cols, lags=[1, 3, 7], windows=[3, 7]):
    for col in base_cols:
        for lag in lags:
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)
        for window in windows:
            df[f"{col}_roll_mean_{window}"] = df[col].rolling(window).mean()
            df[f"{col}_roll_std_{window}"] = df[col].rolling(window).std()
    return df

key_features = ["sp500_close", "vix_close", "bitcoin_close", "oil_close"]
all_data = add_lag_rolling_features(all_data, key_features)
all_data.dropna(inplace=True)
all_data = all_data.loc[:, all_data.nunique() > 1]
logging.info("Final dataset shape with lags/rolls: %s", all_data.shape)

target = "sp500_close"
exclude_cols = ["timestamp", target]
exog_vars = [col for col in all_data.columns if col not in exclude_cols]

results = []
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2025-03-01")
current_date = start_date

while current_date <= end_date:
    logging.info(f"Backtest for month starting {current_date.strftime('%Y-%m-%d')}")
    next_month = current_date + pd.DateOffset(months=1)

    train = all_data[all_data["timestamp"] < current_date]
    test = all_data[(all_data["timestamp"] >= current_date) & (all_data["timestamp"] < next_month)]

    if len(train) < 100 or len(test) < 10:
        logging.warning("Skipping month due to insufficient data")
        current_date += pd.DateOffset(months=1)
        continue

    y_train = train[target]
    y_test = test[target]
    X_train = train[exog_vars]
    X_test = test[exog_vars]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    best_aic = np.inf
    best_model = None
    best_order = None
    best_seasonal_order = None

    for p in [0, 1]:
        for q in [0, 1]:
            for P in [0, 1]:
                for Q in [0, 1]:
                    try:
                        model = SARIMAX(
                            y_train,
                            exog=X_train_pca,
                            order=(p, 1, q),
                            seasonal_order=(P, 0, Q, 12),
                            enforce_stationarity=False,
                            enforce_invertibility=False
                        )
                        model_fit = model.fit(disp=False)
                        if model_fit.aic < best_aic:
                            best_aic = model_fit.aic
                            best_model = model_fit
                            best_order = (p, 1, q)
                            best_seasonal_order = (P, 0, Q, 12)
                    except:
                        continue

    if best_model:
        forecast = best_model.forecast(steps=len(X_test_pca), exog=X_test_pca)
        r2 = r2_score(y_test, forecast)
        mae = mean_absolute_error(y_test, forecast)
        rmse = np.sqrt(mean_squared_error(y_test, forecast))

        results.append({
            "Backtest": len(results) + 1,
            "Train Size": len(train),
            "Test Start": current_date.strftime("%Y-%m-%d"),
            "R2": r2,
            "MAE": mae,
            "RMSE": rmse,
            "Order": best_order,
            "Seasonal Order": best_seasonal_order,
            "AIC": best_aic
        })

    current_date += pd.DateOffset(months=1)

results_df = pd.DataFrame(results)
os.makedirs("results", exist_ok=True)
results_df.to_csv("results/sarimax_backtest_lags_rolls.csv", index=False)
print(results_df)

2025-04-14 22:55:00,849 - INFO - Loading bitcoin.csv
2025-04-14 22:55:00,858 - INFO - Loading gold.csv
2025-04-14 22:55:00,861 - INFO - Loading google_trends.csv
2025-04-14 22:55:00,864 - INFO - Loading sp500.csv
2025-04-14 22:55:00,867 - INFO - Loading treasury_3m.csv
2025-04-14 22:55:00,870 - INFO - Loading treasury_10y.csv
2025-04-14 22:55:00,872 - INFO - Loading copper.csv
2025-04-14 22:55:00,877 - INFO - Loading oil.csv
2025-04-14 22:55:00,879 - INFO - Loading unemployment.csv
2025-04-14 22:55:00,883 - INFO - Loading MOEX.csv
2025-04-14 22:55:00,886 - INFO - Loading SSE.csv
2025-04-14 22:55:00,888 - INFO - Loading STOXX_600.csv
2025-04-14 22:55:00,890 - INFO - Loading vix.csv
2025-04-14 22:55:00,892 - INFO - Loading spgsci.csv
2025-04-14 22:55:00,896 - INFO - Merging all dataframes
2025-04-14 22:55:00,916 - INFO - Final dataset shape with lags/rolls: (794, 69)
2025-04-14 22:55:00,917 - INFO - Backtest for month starting 2023-01-01
2025-04-14 22:55:10,644 - INFO - Backtest for mont

    Backtest  Train Size  Test Start        R2        MAE       RMSE  \
0          1         243  2023-01-01  0.639884   3.567644   4.696596   
1          2         263  2023-02-01  0.442231   3.951232   4.944281   
2          3         282  2023-03-01  0.507930   3.539787   4.126693   
3          4         305  2023-04-01  0.345633   1.749195   2.411999   
4          5         324  2023-05-01 -1.363027   5.319201   6.009797   
5          6         346  2023-06-01 -2.867095  10.077607  10.697305   
6          7         367  2023-07-01 -2.363279  10.124038  11.195012   
7          8         387  2023-08-01  0.773276   2.056073   2.582181   
8          9         410  2023-09-01  0.819497   2.940014   3.682682   
9         10         430  2023-10-01  0.562876   3.784457   5.038851   
10        11         452  2023-11-01  0.926585   2.078619   2.679495   
11        12         473  2023-12-01  0.823805   2.282809   3.133692   
12        13         493  2024-01-01 -1.169285   9.302325  10.66